In [14]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/low'
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'

In [15]:
predictions = pd.read_pickle(os.path.join(dir_, 'post_filtering_old', str(0.005) + '_prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

In [16]:
predictions[:5]

,uid,tid,rating
20,0,125,2.265003
32,0,177,2.514423
50,0,282,2.146869
61,0,350,2.423719
94,0,542,2.223415


In [17]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
7,0,3,0.25,1
11,0,7,2.50,3
32,0,18,1.50,2
34,0,20,2.50,3
45,0,31,5.00,17


In [18]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
    print(len(groups[0]))

2
3
4
5
6
7
8


In [19]:
num_user = len(predictions['uid'].unique())
num_user

220

In [20]:
precision_4 = []
precision_20 = []
precision_50 = []

recall_4 = []
recall_20 = []
recall_50 = []

In [21]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_precisions = []
    avg_recalls = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        recalls =[]
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                high_rating = 0
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        
                precision = high_rating / top_n_size
                recall = high_rating / len(test[test['uid'] == uid])
                precisions.append(precision)
                recalls.append(recall)
                
        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
        
        avg_recall = 0
        for recall in recalls:
            avg_recall += recall
        avg_recall /= len(recalls)
        avg_recalls.append(avg_recall)
        
    if top_n_size == 4:
        precision_4.append(avg_precisions)
    elif top_n_size == 20:
        precision_20.append(avg_precisions)
    elif top_n_size == 50:
        precision_50.append(avg_precisions)
        
    if top_n_size == 4:
        recall_4.append(avg_recalls)
    elif top_n_size == 20:
        recall_20.append(avg_recalls)
    elif top_n_size == 50:
        recall_50.append(avg_recalls)


In [22]:
for i in precision_4:
    print(i)

for i in precision_20:
    print(i)

for i in precision_50:
    print(i)

for i in recall_4:
    print(i)

for i in recall_20:
    print(i)

for i in recall_50:
    print(i)

[0.21, 0.1414141414141414, 0.11842105263157894, 0.07692307692307693, 0.09166666666666666, 0.08928571428571429, 0.05803571428571429]
[0.1470000000000001, 0.08636363636363634, 0.0730263157894736, 0.051538461538461526, 0.052499999999999984, 0.05178571428571428, 0.03660714285714287]
[0.1178666666666665, 0.07515151515151508, 0.06605263157894734, 0.04984615384615387, 0.04633333333333335, 0.046428571428571444, 0.033571428571428585]
[0.002018533997219881, 0.002177801093612741, 0.0021167177382649983, 0.0014892640730984833, 0.002035257928516486, 0.0020550766817816137, 0.0012528805728839465]
[0.006808949144441707, 0.005839466237025804, 0.006018669768769152, 0.005771988685513573, 0.006393906830466243, 0.005405267685066041, 0.003477726634574832]
[0.013215431094349337, 0.012162562479073486, 0.01296867352889465, 0.012332016159097351, 0.012429583388226782, 0.011336338080231528, 0.009699153395450442]


In [23]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [24]:
nDCG_4 = []
nDCG_20 = []
nDCG_50 = []

In [25]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_nDCGs = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        nDCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                truth_rating = []
                uid = group[k]
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    
                    if len(t) > 0:
                        truth_rating.append(t.iloc[0]['rating'])
                    else:
                        truth_rating.append(0)
                max_rating = test[test['uid']==uid].sort_values(by=['rating'],  ascending=False)['rating'].values[:top_n_size]
                nDCG = ndcg_at_k(truth_rating, max_rating, top_n_size, method=1)
                nDCGs.append(nDCG)                
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)
        avg_nDCGs.append(avg_nDCG)
        
    if top_n_size == 4:
        nDCG_4.append(avg_nDCGs)
    elif top_n_size == 20:
        nDCG_20.append(avg_nDCGs)
    elif top_n_size == 50:
        nDCG_50.append(avg_nDCGs)

In [26]:
for i in nDCG_4:
    print(i)

for i in nDCG_20:
    print(i)

for i in nDCG_50:
    print(i)

[0.11320742830877452, 0.05791563633974507, 0.05229892267577806, 0.02729102744159426, 0.027695938796609244, 0.027019384811496, 0.022787618512517177]
[0.07983091617926148, 0.039263058105305955, 0.03607530597555705, 0.024368659070881992, 0.019587963261293306, 0.02635809026899343, 0.01914253911744192]
[0.06587081575711434, 0.036560215127179156, 0.034123623925083016, 0.026259836235324276, 0.021016214280814752, 0.027147513553457094, 0.01876477455948753]
